<a href="https://colab.research.google.com/github/A01720548/TC2008B.4-ActividadIntegradora/blob/main/ActividadIntegradoraFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esteban de la Maza A01720548

In [1]:
!pip3 install mesa

In [14]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128
import numpy as np
import pandas as pd
import time
import datetime
import signal
import math

In [15]:
apiladas = 0
nCajas = 0
nPilas = 0
nPila = 0
cantApiladas = 0

def recopilacion(K):
  global nCajas
  global nPilas
  nCajas = K
  nPilas = math.floor(K/5)


def obtenerMatriz(model):

    matriz = np.zeros((model.matriz.width, model.matriz.height))
    for celda in model.matriz.coord_iter():
        contenidoCelda, x, y = celda
        for agent in contenidoCelda:
            matriz[x][y] = agent.height
    return matriz

class Robot(Agent):
    def __init__(self, unique_id, model, height):
      super().__init__(unique_id, model)
      self.height = height
      self.boolCaja = 0
      self.caja = -1


    def apilarCaja(self):
      global cantApiladas
      global apiladas
      global nPila
      b = caja(self.pos, self, cantApiladas + 1)
      b.apilada = True
      self.model.matriz.place_agent(b, self.pos)
      cantApiladas +=1
      self.boolCaja = 2
      apiladas +=1
      if cantApiladas == 5:
        nPila +=1
        cantApiladas = 0


    def movimiento(self, pos):
      neighbours = self.model.matriz.get_neighbors(
          self.pos,
          moore=False,
          include_center=False)
      for neighbour in neighbours:
        if (isinstance(neighbour, Robot) and neighbour.pos == pos):
          return False
      return True         
    

    def encontrarCaja(self):
      neighbours = self.model.matriz.get_neighbors(
          self.pos,
          moore=False,
          include_center=True)
      for neighbour in neighbours:
        if (isinstance(neighbour, caja)):
          if (neighbour.height) == 1 and neighbour.apilada == False:
            return neighbour
      return -1 

          
    def movimientoPila(self) :
      x, y = self.pos
      if x != self.model.width/2 and self.movimiento((x+1,y)):
        self.model.matriz.move_agent(self, (x+1,y))
      if y != self.model.height/2 and self.movimiento((x,y+1)):
        self.model.matriz.move_agent(self, (x,y+1))
      if x==self.model.width/2 and y==self.model.height/2:
        self.boolCaja = 0  

    def moverAApilado(self):
      global nPila
      x, y = self.pos
      if x != 0 and self.movimiento((x-1,y)):
        self.model.matriz.move_agent(self, (x-1,y))
      if y < nPila and self.movimiento((x,y+1)):
        self.model.matriz.move_agent(self, (x,y+1))
      if y > nPila and self.movimiento((x,y-1)):
        self.model.matriz.move_agent(self, (x,y-1))
      if x==0 and y==nPila:
        self.apilarCaja()


    def rMovimiento(self):
      siguientesMovimientos = self.model.matriz.get_neighborhood(self.pos, False, True)
      sMovimiento = self.random.choice(siguientesMovimientos)
      # Realizar el movimiento
      xS, yS = sMovimiento
      x, y = self.pos

      if self.movimiento(sMovimiento):
        if (x == 0 and xS == self.model.height - 1):
          return
        if (y == 0 and yS == self.model.width - 1):
          return
        if (x == self.model.height - 1 and xS == 0):
          return
        if (y == self.model.width - 1 and yS == 0):
          return    
        self.model.matriz.move_agent(self, sMovimiento)



    def step(self):
      if self.boolCaja == 0:
        self.caja = self.encontrarCaja()
        if self.caja != -1:             
          self.boolCaja = 1
          self.model.matriz.remove_agent(self.caja)
        

    def advance(self):
      if self.boolCaja == 1:
        self.moverAApilado()
      elif self.boolCaja == 2:
        self.movimientoPila()
      else:
        self.rMovimiento()



class caja(Agent):

    def __init__(self, unique_id, model, height):
      super().__init__(unique_id, model)
      self.height = height
      self.apilada = False
      

class RobotModel(Model):

    def __init__(self, width, height, K):
      self.matriz = MultiGrid(width, height, False)
      self.schedule = SimultaneousActivation(self)
      self.numCajas = K
      self.width = width
      self.height = height
      global nCajas
      recopilacion(K)
      tMatriz = width*height
      # print(total)
      posiciones = np.zeros((width,height), int)
      x = 0
      while x < K:
        pos = np.random.randint(0,width-1)
        pos2 = np.random.randint(0,height-1)
        if posiciones[pos][pos2] == 0:
          posiciones[pos][pos2] = 1
          x+=1

      y = 0
      while y < 5:
        pos = np.random.randint(0,width-1)
        pos2 = np.random.randint(0,height-1)
        if posiciones[pos][pos2] == 0:
          posiciones[pos][pos2] = 2
          y+=1
      for (content, x, y) in self.matriz.coord_iter():
        if posiciones[x][y] == 2:
          a = Robot((x, y), self, 6)
          self.matriz.place_agent(a, (x,y)) 
          self.schedule.add(a)
        if posiciones[x][y] == 1:
          b = caja((x, y), self, 1)
          self.matriz.place_agent(b, (x,y))
          self.schedule.add(b)

      self.datacollector = DataCollector(
          model_reporters={"matriz": obtenerMatriz})
    
    
    def completo(self):
      self.porApiladas = apiladas / self.numCajas
      if self.porApiladas == 1 :
        return 1
      else:
        return self.porApiladas

    
    def step(self):
      self.datacollector.collect(self)
      self.schedule.step()



In [16]:
tamanyoMatriz = 6 #Dimensión de la matriz
K = 15 #Cantidad de cajas
tInicio = time.time()
tMax = .1 #Tiempo maximo de ejecución

contMovimientos = 0

model = RobotModel(tamanyoMatriz, tamanyoMatriz, K)
# and model.allClean() != 1
while(time.time() - tInicio) < tMax and model.completo() != 1:
  contMovimientos+=1
  model.step()

contMovimientos+=1
model.step()

print('%Cajas apiladas: ', str(model.completo() * 100), '%')
print('Tiempo:', str(datetime.timedelta(seconds=(time.time() - tInicio))))
print('Movimientos realizados: ', contMovimientos)

Tiempo: 0:00:00.012966
%Cajas apiladas:  100 %
Movimientos realizados:  78


In [17]:
matrizTotal = model.datacollector.get_model_vars_dataframe()


In [18]:
%%capture

fig, axs = plt.subplots(figsize=(4,4))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(matrizTotal.iloc[0][0], cmap=plt.cm.Blues)

def animate(i):
    patch.set_data(matrizTotal.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=contMovimientos)

In [19]:

anim